#### 从数据库导入数据

In [1]:
import pymysql.cursors
import pandas as pd

db = pymysql.connect(host='localhost',user='root',password='root',db='lagou',charset='utf8',cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()
sql = 'SELECT * FROM dataset;'
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(list(result))
db.close()

df

,jobNature,companySize,workYear,education,industryField,thirdType,salaryMin,salaryMax,nameUnfit,positionId,...,financeStage,firstType,secondType,district,salary,resumeProcessRate,resumeProcessDay,score,famousCompany,isFocus
0,全职,500-2000人,1-3年,本科,旅游,数据分析,18,35,0,6112946,...,D轮及以上,开发|测试|运维类,数据开发,福田区,18k-35k,100,1,199,0,1
1,全职,500-2000人,3-5年,本科,"消费生活,移动互联网",商业,15,25,0,6229664,...,C轮,产品|需求|项目类,数据分析,南山区,15k-25k,100,1,179,0,1
2,全职,2000人以上,5-10年,本科,金融,建模,15,30,0,6853427,...,上市公司,开发|测试|运维类,数据开发,福田区,15k-30k,100,1,130,1,0
3,全职,2000人以上,1-3年,本科,金融,Hadoop,20,30,0,6692363,...,不需要融资,开发|测试|运维类,数据开发,南山区,20k-30k,100,2,89,0,0
4,全职,2000人以上,1-3年,大专,"移动互联网,游戏",数据分析,8,10,0,6835783,...,不需要融资,产品|需求|项目类,数据分析,南山区,8k-10k,0,1,69,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,全职,2000人以上,3-5年,本科,社交,产品经理,15,25,1,5276304,...,上市公司,产品|需求|项目类,产品经理,南山区,15k-25k,100,1,10,1,0
271,全职,2000人以上,3-5年,本科,社交,数据挖掘,18,35,0,6842800,...,上市公司,开发|测试|运维类,数据开发,南山区,18k-35k,100,1,10,1,1
272,全职,2000人以上,3-5年,本科,社交,数据分析,20,40,0,5250466,...,上市公司,产品|需求|项目类,数据分析,南山区,20k-40k,100,1,10,1,1
273,全职,2000人以上,1-3年,本科,金融,数据分析,20,30,0,6391267,...,未融资,开发|测试|运维类,数据开发,南山区,20k-30k,15,1,4,1,0


#### 拆分训练集和测试集

In [2]:
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1]
X = pd.get_dummies(X).values
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
X_names = pd.get_dummies(df.iloc[:,:-1]).columns
X_names

Index(['salaryMin', 'salaryMax', 'nameUnfit', 'resumeProcessRate',
       'resumeProcessDay', 'score', 'famousCompany', 'jobNature_全职',
       'jobNature_兼职', 'jobNature_实习',
       ...
       'salary_7k-13k', 'salary_7k-14k', 'salary_8k-10k', 'salary_8k-11k',
       'salary_8k-12k', 'salary_8k-13k', 'salary_8k-15k', 'salary_8k-16k',
       'salary_9k-12k', 'salary_9k-14k'],
      dtype='object', length=687)

#### 网格搜索（含交叉验证）——找出最优参数

In [8]:
?GridSearchCV

In [5]:
?GradientBoostingClassifier

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

rfc = GradientBoostingClassifier()
parameters = {
    'random_state':[0],
    'loss':['deviance', 'exponential'],
    'learning_rate':[0.05, 0.1, 0.2],
    'n_estimators':[50,100,300],
    'min_samples_leaf':[3,4,5,6,7],
    'max_depth':[7,8,9,10]
}
scorings = ['recall','roc_auc']
for scoring in scorings:
    print('===========================================================================')
    print('此次调参的scoring为：%s' % scoring)
    print()
    clf = GridSearchCV(rfc, parameters, scoring=scoring, cv=10)
    clf.fit(X_train, y_train)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    print('【交叉验证】过程如下：')
    for mean, std, param in zip(means, stds, params):
        print('%s得分为 %0.3f(+-%0.3f)时，参数为：%r' % (scoring, mean, std * 2, param))
    print()
    print('【训练集】最佳参数为：%r' % clf.best_params_)
    print('【训练集】最佳参数得分为：%0.3f' % clf.best_score_)
    print()
    print('【测试集】%s得分为：%0.3f' % (scoring, clf.score(X_test, y_test)))    # 直接获取测试集的得分
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    reports = classification_report(y_true, y_pred)
    print('【测试集中实际值与预测值对比】模型报告如下：')
    print(reports)

此次调参的scoring为：recall

【交叉验证】过程如下：
recall得分为 0.680(+-0.580)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 3, 'n_estimators': 50, 'random_state': 0}
recall得分为 0.720(+-0.607)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 3, 'n_estimators': 100, 'random_state': 0}
recall得分为 0.805(+-0.483)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 3, 'n_estimators': 300, 'random_state': 0}
recall得分为 0.725(+-0.554)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 4, 'n_estimators': 50, 'random_state': 0}
recall得分为 0.765(+-0.478)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 4, 'n_estimators': 100, 'random_state': 0}
recall得分为 0.785(+-0.498)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 4, 'n_estimators': 300, 'random_state': 0}
recall得分为 0.785(+-0.529)时，参数为：{'learning_rate': 0.

【交叉验证】过程如下：
roc_auc得分为 0.963(+-0.074)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 3, 'n_estimators': 50, 'random_state': 0}
roc_auc得分为 0.974(+-0.074)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 3, 'n_estimators': 100, 'random_state': 0}
roc_auc得分为 0.979(+-0.039)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 3, 'n_estimators': 300, 'random_state': 0}
roc_auc得分为 0.975(+-0.043)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 4, 'n_estimators': 50, 'random_state': 0}
roc_auc得分为 0.974(+-0.056)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 4, 'n_estimators': 100, 'random_state': 0}
roc_auc得分为 0.971(+-0.077)时，参数为：{'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 4, 'n_estimators': 300, 'random_state': 0}
roc_auc得分为 0.975(+-0.060)时，参数为：{'learning_rate': 0.05, 'loss': 'de